In [ ]:
!pip install tensorflow tensorflow-hub tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 9.6 MB/s eta 0:00:00


# **Importing Libraries and Datasets**


In [ ]:
# Importing the used libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive


In [ ]:
drive.mount('/content/Files',force_remount=True) # Getting all files from my google drive account
drive_path = '/content/Files/MyDrive/Deep Bonus Project/' # The path of the files on this colab

Mounted at /content/Files


# **Egyptian Sentiment Classification Model**

In [ ]:
# Load the data
file_path = drive_path+'egyptian_dataset.csv'
data = pd.read_csv(file_path)

# Shuffle the DataFrame
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Splitting Sentences and labels
sentences = data['sentence'].values
labels = data['label'].values  # Labels are be 0 or 1

# Split the data into training and testing sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

# Tokenize the sentences
# Out Of Vocab tokenizer: giving each word in the dataset an index, and if there is new word appeared in test it gets <OOV> token which is usually at index 0 or 1
# for example on training dataset: oov: {<OOV>:1,is:2,cat:3,beutiful:4}
# fot this sentence when converted to sequence: The cat is beutifull: [1,3,2,4]

tokenizer = Tokenizer(oov_token='<OOV>') # Defining the tokenizer
tokenizer.fit_on_texts(train_sentences)  # fitting the tokenizer on the training data

# Convert sentences to sequences of numbers
train_sequences = tokenizer.texts_to_sequences(train_sentences) # Using the tokenizer to covert from the sentences to sequences of indexes like: [1,3,2,4]
test_sequences = tokenizer.texts_to_sequences(test_sentences) # Doing the same but on test data

# Pad sequences to make them of the same length
max_length = max(len(seq) for seq in train_sequences) # Getting the maximum length of a sentence
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post') # Applying padding of zeros (post method) to make all of the sequences of the same length
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post') # Doing the same on test dataset

# Build the RNN model
egyptian_model = Sequential() # Sequential model is a model of sequenced layers
# Embedding layer
# Input_Dim: size of vocabulary (unique words in the dataset)
# Output_Dim: Embedding each word in a 16-dimensional vector
# input_length: the length of the input sequences (which is the maximum length of all sentences)
egyptian_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_length=max_length))
# Adding LSTM layer with 64 memory cell
egyptian_model.add(LSTM(64))
# Applying sigmoid to do classification
egyptian_model.add(Dense(1, activation='sigmoid'))

# Compile the model
egyptian_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Splitting Sentences and labels
sentences = data['sentence'].values
labels = data['label'].values  # Labels are be 0 or 1


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

egyptian_model_history = egyptian_model.fit(train_padded, train_labels, epochs=8, validation_data=(test_padded, test_labels), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = egyptian_model.evaluate(test_padded, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/8
16/16 [==============================] - 4s 54ms/step - loss: 0.6936 - accuracy: 0.5040 - val_loss: 0.6927 - val_accuracy: 0.8031
Epoch 2/8
16/16 [==============================] - 0s 26ms/step - loss: 0.6901 - accuracy: 0.6304 - val_loss: 0.6768 - val_accuracy: 0.8740
Epoch 3/8
16/16 [==============================] - 0s 29ms/step - loss: 0.4489 - accuracy: 0.9308 - val_loss: 0.2285 - val_accuracy: 0.9134
Epoch 4/8
16/16 [==============================] - 0s 26ms/step - loss: 0.0833 - accuracy: 0.9802 - val_loss: 0.4666 - val_accuracy: 0.8819
Epoch 5/8
4/4 [==============================] - 0s 8ms/step - loss: 0.2285 - accuracy: 0.9134
Test Accuracy: 91.34%


In [ ]:
egyptian_model.save(drive_path+'egyptian_model_8Epochs.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
egyptian_model = load_model(drive_path+'egyptian_model_8Epochs.h5')

In [ ]:
def predict_sentiment(input_sentence):
    # Tokenize and pad the input sentence
    input_sequence = tokenizer.texts_to_sequences([input_sentence])
    padded_input = pad_sequences(input_sequence, maxlen=max_length, padding='post')

    # Make predictions
    predictions = egyptian_model.predict(padded_input)

    # Convert the prediction to a sentiment label
    sentiment_label = 'postivie' if predictions[0][0] > 0.5 else 'negative'

    return sentiment_label


In [ ]:
# Example usage
input_sentence = input("Enter arabic sentence: ")
predicted_sentiment = predict_sentiment(input_sentence)
print(f"Predicted Sentiment: {predicted_sentiment}")

Enter arabic sentence: البورصة المصرية في افضل احوالها
1/1 [==============================] - 0s 30ms/step
Predicted Sentiment: postivie


# **French Sentiment Classification Model**

In [ ]:
# Load the data
file_path = drive_path+'french_dataset.csv'
data = pd.read_csv(file_path)

# Shuffle the DataFrame
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Preprocess the data
sentences = data['sentence'].values
labels = data['label'].values  # Labels are be 0 or 1

# Split the data into training and testing sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

# Tokenize the sentences
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)

# Convert sentences to sequences of numbers
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

# Pad sequences to make them of the same length
max_length = max(len(seq) for seq in train_sequences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Build the RNN model
french_model = Sequential()
french_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_length=max_length))
french_model.add(LSTM(64))
french_model.add(Dense(64, activation='relu'))
french_model.add(Dropout(0.5))
french_model.add(Dense(1, activation='sigmoid'))

# Compile the model
french_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

french_model_history = french_model.fit(train_padded, train_labels, epochs=8, validation_data=(test_padded, test_labels), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = french_model.evaluate(test_padded, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/8
250/250 [==============================] - 9s 28ms/step - loss: 0.6934 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.4950
Epoch 2/8
250/250 [==============================] - 8s 32ms/step - loss: 0.6949 - accuracy: 0.5041 - val_loss: 0.6924 - val_accuracy: 0.5100
Epoch 3/8
250/250 [==============================] - 6s 25ms/step - loss: 0.6056 - accuracy: 0.6862 - val_loss: 0.5737 - val_accuracy: 0.7075
Epoch 4/8
250/250 [==============================] - 8s 34ms/step - loss: 0.4107 - accuracy: 0.8288 - val_loss: 0.5741 - val_accuracy: 0.7075
Epoch 5/8
250/250 [==============================] - 6s 25ms/step - loss: 0.2891 - accuracy: 0.8978 - val_loss: 0.6941 - val_accuracy: 0.7060
Epoch 6/8
63/63 [==============================] - 1s 8ms/step - loss: 0.5737 - accuracy: 0.7075
Test Accuracy: 70.75%


In [ ]:
french_model.save(drive_path+'french_model_8Epochs.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
french_model = load_model(drive_path+'french_model_8Epochs.h5')

In [ ]:
def predict_sentiment(input_sentence):
    # Tokenize and pad the input sentence
    input_sequence = tokenizer.texts_to_sequences([input_sentence])
    padded_input = pad_sequences(input_sequence, maxlen=max_length, padding='post')

    # Make predictions
    predictions = french_model.predict(padded_input)

    # Convert the prediction to a sentiment label
    sentiment_label = 'postivie' if predictions[0][0] > 0.5 else 'negative'

    return sentiment_label


In [ ]:
# Example usage
input_sentence = input("Enter french sentence: ")
predicted_sentiment = predict_sentiment(input_sentence)
print(f"Predicted Sentiment: {predicted_sentiment}")

Enter french sentence: C'est comme si mon cœur se brisait de tristesse
1/1 [==============================] - 0s 32ms/step
Predicted Sentiment: negative
